<a href="https://colab.research.google.com/github/francoisdoanp/MLTBP/blob/master/Project_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Machine learning - Final Project**

# Turbofan engine degradation dataset (NASA)

# Data Preparation

**Importing necessary packages**



In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
import math

**Importing the Turbofan engine degradation dataset.**

**Files are located in the following Github repository: https://github.com/francoisdoanp/MLTBP**

We have 4 training datasets, which contains information about one hundred engines, all of the same type. Thus, we will combine the training and test data sets. 

The training and test sets have 21 columns: ID, Time (Cycles), 3 columns for operational settings and 21 sensor measurements.

The training and testing sets have the same format, while the validation sets only contain the real RUL (remaining useful life).

For more information on the data, consult the read me at the following address:https://github.com/francoisdoanp/MLTBP/blob/master/readme.txt

In [8]:
url_base = 'https://raw.githubusercontent.com/francoisdoanp/MLTBP/master/'

file_train_1 = 'train_FD001.txt'
file_train_2 = 'train_FD002.txt'
file_train_3 = 'train_FD003.txt'
file_train_4 = 'train_FD004.txt'

file_test_1 = 'test_FD001.txt'
file_test_2 = 'test_FD002.txt'
file_test_3 = 'test_FD003.txt'
file_test_4 = 'test_FD004.txt'

file_valid_1 = 'RUL_FD001.txt'
file_valid_2 = 'RUL_FD002.txt'
file_valid_3 = 'RUL_FD003.txt'
file_valid_4 = 'RUL_FD004.txt'


pt1 = pd.read_csv(url_base + file_train_1, sep=' ', header=None)
pt2 = pd.read_csv(url_base + file_train_2, sep=' ', header=None)
pt3 = pd.read_csv(url_base + file_train_3, sep=' ', header=None)
pt4 = pd.read_csv(url_base + file_train_4, sep=' ', header=None)

pte1 = pd.read_csv(url_base + file_test_1, sep=' ', header=None)
pte2 = pd.read_csv(url_base + file_test_2, sep=' ', header=None)
pte3 = pd.read_csv(url_base + file_test_3, sep=' ', header=None)
pte4 = pd.read_csv(url_base + file_test_4, sep=' ', header=None)

pv1 = pd.read_csv(url_base + file_valid_1, header=None)
pv2 = pd.read_csv(url_base + file_valid_2, header=None)
pv3 = pd.read_csv(url_base + file_valid_3, header=None)
pv4 = pd.read_csv(url_base + file_valid_4, header=None)


# Updating unit numbers (1416 motor in total)
#Labeled event(failure) data
pt2[0] = pt2[0].apply(lambda x: x+100)
pt3[0] = pt3[0].apply(lambda x: x+360)
pt4[0] = pt4[0].apply(lambda x: x+460)

#Censored data (no event)
pte1[0] = pte1[0].apply(lambda x: x+708)
pte2[0] = pte2[0].apply(lambda x: x+808)
pte3[0] = pte3[0].apply(lambda x: x+1067)
pte4[0] = pte4[0].apply(lambda x: x+1167)


# Joining the dataframes
train_pd = pd.concat([pt1,pt2,pt3,pt4])
test_pd = pd.concat([pte1,pte2,pte3,pte4])
valid_pd = pd.concat([pv1,pv2,pv3,pv4], ignore_index=True)

train_pd = train_pd.drop(train_pd.columns[[26,27]], axis='columns')
test_pd = test_pd.drop(test_pd.columns[[26,27]], axis='columns')

#Survival_pd = pd.concat([train_pd,test_pd])


# Assigning labels to Dataframe's columns based on the Readme

train_pd.columns = ['Unit Number', 'Time (Cycles)', 'OS1', 'OS2', 'OS3', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21']
test_pd.columns = ['Unit Number', 'Time (Cycles)', 'OS1', 'OS2', 'OS3', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21']
#Survival_pd.columns = ['Unit Number', 'Time (Cycles)', 'OS1', 'OS2', 'OS3', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21']
valid_pd.columns = ['RUL']

# Keep only the last cycle of each unit numbers
#Survival_pd = Survival_pd.groupby('Unit Number').max()

#display(Survival_pd)

#Loading scaler

scaler = StandardScaler()
  

Time (Cycles)      OS1     OS2    OS3      S1      S2       S3  \
Unit Number                                                                   
1                      192   0.0047  0.0005  100.0  518.67  644.21  1605.44   
2                      287   0.0076  0.0006  100.0  518.67  643.94  1610.10   
3                      179   0.0058  0.0005  100.0  518.67  643.93  1606.50   
4                      189   0.0059  0.0006  100.0  518.67  644.53  1612.11   
5                      269   0.0055  0.0005  100.0  518.67  644.02  1609.41   
6                      188   0.0044  0.0005  100.0  518.67  644.12  1607.63   
7                      259   0.0068  0.0005  100.0  518.67  644.27  1607.70   
8                      150   0.0053  0.0005  100.0  518.67  644.02  1610.35   
9                      201   0.0050  0.0005  100.0  518.67  644.04  1607.04   
10                     222   0.0063  0.0005  100.0  518.67  644.35  1613.62   
11                     240   0.0065  0.0005  100.0  518.67  644.09  1609.02   
12                     170   0.0072  0.0005  100.0  518.67  643.91  1607.10   
13                     163   0.0064  0.0005  100.0  518.67  644.26  1612.88   
14                     180   0.0070  0.0005  100.0  518.67  644.17  1609.93   
15                     207   0.0055  0.0005  100.0  518.67  643.93  1608.43   
16                     209   0.0074  0.0005  100.0  518.67  644.47  1608.26   
17                     276   0.0051  0.0005  100.0  518.67  644.09  1608.86   
18                     195   0.0071  0.0005  100.0  518.67  644.05  1609.52   
19                     158   0.0051  0.0005  100.0  518.67  644.12  1605.58   
20                     234   0.0057  0.0005  100.0  518.67  644.12  1608.02   
21                     195   0.0064  0.0004  100.0  518.67  644.03  1606.47   
22                     202   0.0057  0.0005  100.0  518.67  643.87  1609.56   
23                     168   0.0051  0.0005  100.0  518.67  644.07  1604.68   
24                     147   0.0067  0.0004  100.0  518.67  643.96  1607.24   
25                     230   0.0060  0.0006  100.0  518.67  644.18  1608.49   
26                     199   0.0056  0.0005  100.0  518.67  644.04  1607.92   
27                     156   0.0050  0.0004  100.0  518.67  644.00  1607.67   
28                     165   0.0053  0.0005  100.0  518.67  644.01  1605.08   
29                     163   0.0073  0.0004  100.0  518.67  644.10  1606.95   
30                     194   0.0058  0.0005  100.0  518.67  644.01  1604.26   
...                    ...      ...     ...    ...     ...     ...      ...   
1386                   110  42.0078  0.8420  100.0  518.67  643.48  1600.09   
1387                    65  42.0078  0.8419  100.0  518.67  642.31  1592.63   
1388                   176  42.0075  0.8420  100.0  518.67  642.82  1591.39   
1389                   178  42.0073  0.8419  100.0  518.67  642.65  1590.57   
1390                   305  42.0079  0.8420  100.0  518.67  642.91  1594.52   
1391                   152  42.0078  0.8418  100.0  518.67  643.60  1601.23   
1392                    39  42.0076  0.8420  100.0  518.67  642.88  1587.32   
1393                    59  42.0075  0.8420  100.0  518.67  642.70  1587.92   
1394                    62  42.0080  0.8417  100.0  518.67  642.80  1588.66   
1395                   126  42.0077  0.8419  100.0  518.67  642.55  1585.41   
1396                    20  42.0059  0.8410  100.0  518.67  642.29  1591.22   
1397                   157  42.0075  0.8418  100.0  518.67  642.69  1593.19   
1398                   230  42.0079  0.8420  100.0  518.67  643.30  1599.35   
1399                   370  42.0079  0.8419  100.0  518.67  642.94  1595.02   
1400                   111  42.0077  0.8417  100.0  518.67  643.28  1596.52   
1401                   179  42.0075  0.8420  100.0  518.67  643.43  1600.65   
1402                   362  42.0078  0.8420  100.0  518.67  643.64  1596.69   
1403                   176  42.0080  0.8419  100.0  518.67  642

**Adding variables Conditons and fault mode**

Note:

**Condition (ONE)** and **Fault ONE** are binary variables.

When Condition(ONE) = 1 (true), it means that the condition is at Sea Level

When Condition(ONE) = 0 (false), it means NO, the condition IS NOT AT SEA LEVEL, and thus is the  second condition; SIX.

When Fault ONE = 1 (true), it means that the fault modes is one (HPC Degradation)

When Fault ONE = 0 (false), it means that the fault mode is TWO (HPC Degradation and Fan degradation)

##### Adding variables Condition and fault modes

def value_condition_train(row):
  if (row['Unit Number'] <= 100):
    return 1
  elif (row['Unit Number'] <= 360) & (row['Unit Number'] > 100):
    return 0
  elif (row['Unit Number'] <= 460) & (row['Unit Number'] > 360):
    return 1
  else:
    return 0
  
def value_fault_train(row):
  if (row['Unit Number'] <= 100):
    return 1
  elif (row['Unit Number'] <= 360) & (row['Unit Number'] > 100):
    return 1
  elif (row['Unit Number'] <= 460) & (row['Unit Number'] > 360):
    return 0
  else:
    return 0

def value_event_train(row):
  return 1
  
def value_condition_test(row):
  if (row['Unit Number'] <= 808):
    return 1
  elif (row['Unit Number'] <= 1067) & (row['Unit Number'] > 808):
    return 0
  elif (row['Unit Number'] <= 1077) & (row['Unit Number'] > 1067):
    return 1
  else:
    return 0
  
def value_fault_test(row):
  if (row['Unit Number'] <= 808):
    return 1
  elif (row['Unit Number'] <= 1067) & (row['Unit Number'] > 808):
    return 1
  elif (row['Unit Number'] <= 1167) & (row['Unit Number'] > 1067):
    return 0
  else:
    return 0

def value_event_test(row):
  return 0


train_pd['Condition (One)'] = train_pd.apply(value_condition_train, axis=1)
train_pd['Fault ONE'] = train_pd.apply(value_fault_train,axis=1)
train_pd['Event'] = train_pd.apply(value_event_train,axis=1)

test_pd['Condition (One)'] = test_pd.apply(value_condition_test, axis=1)
test_pd['Fault ONE'] = test_pd.apply(value_fault_test,axis=1)
test_pd['Event'] = test_pd.apply(value_event_test,axis=1)

Survival_pd = pd.concat([train_pd,test_pd])

# Keep only the last cycle of each unit numbers
#Survival_pd = Survival_pd.iloc[Survival_pd.groupby(['Unit Number'])['Time (Cycles)'].idxmax()]

display(Survival_pd)


##### corr = train_pd.groupby(['RUL']).corr()
corr.style.background_gradient(cmap='coolwarm')

**How do we know when an engine fails?**

We look at the last time cycle for every unit number.



In [93]:
train_pd.groupby(['Unit Number'])['Time (Cycles)'].max().hist(bins=30, grid=False)
plt.xlabel('Number of Cycles')
plt.ylabel('Frequency')


NameError: ignored

#**Model 1: Multiple Linear Regression (Not time-sensitive)**






**Preparing data**

In [0]:
# Scaling data

train_pd_scaled = train_pd.copy()
train_pd_scaled.iloc[:,2:26] = scaler.fit_transform(train_pd.iloc[:,2:26])

test_pd_lm = test_pd.copy()
test_pd_lm.iloc[:,2:26] = scaler.fit_transform(test_pd.iloc[:,2:26])

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', -1)



In [0]:
# Removing RUL and Unit Number columns, as we do not want these features to be in our predictors

train_pd_lm = train_pd_scaled.copy()
train_pd_lm = train_pd_lm.drop(['RUL', 'Unit Number'],axis=1)

# Keeping only last time cycle for each unit number

idx = test_pd_lm.groupby(['Unit Number'])['Time (Cycles)'].transform(max) == test_pd_lm['Time (Cycles)']
test_pd_lm = test_pd_scaled[idx]

# Removing Unit Number column

test_pd_lm = test_pd_lm.drop(['Unit Number'], axis=1)

**Fitting Linear Model**

In [103]:
# Fitting Linear model

reg = LinearRegression().fit(train_pd_lm, y_train)

y_pred = reg.predict(test_pd_lm)

acc_score_lm =  metrics.mean_squared_error(valid_pd, y_pred)

print(f'The square root of the mean squared error for the linear model is: {math.sqrt(acc_score_lm)}.')


The square root of the mean squared error for the linear model is: 54.8991731927702.


In [0]:
print(adjusted_error(valid_pd, y_pred))